<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-Library" data-toc-modified-id="Import-Library-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import Library</a></span></li><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load Data</a></span></li><li><span><a href="#EDA" data-toc-modified-id="EDA-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>EDA</a></span><ul class="toc-item"><li><span><a href="#Drop-columns" data-toc-modified-id="Drop-columns-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Drop columns</a></span></li><li><span><a href="#Check-Missing-Value" data-toc-modified-id="Check-Missing-Value-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Check Missing Value</a></span></li></ul></li><li><span><a href="#Preprocessing" data-toc-modified-id="Preprocessing-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Preprocessing</a></span></li><li><span><a href="#Naive-Bayes" data-toc-modified-id="Naive-Bayes-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Naive Bayes</a></span></li><li><span><a href="#Sentiment" data-toc-modified-id="Sentiment-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Sentiment</a></span></li></ul></div>

## Import Library

In [1]:
import numpy as np
import pandas as pd 
import re
import pythainlp
from pythainlp.corpus.common import thai_words
from pythainlp.tokenize import dict_trie, word_tokenize
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import glob
import matplotlib.pyplot as plt
%matplotlib inline

## Load Data

In [2]:
filenames = glob.glob("final/*.csv")

In [3]:
df = pd.DataFrame()

In [4]:
for filename in filenames:
    temp = pd.read_csv(filename)
    company = filename.split("/")[1].split("_")[0]
    temp["company"] = company
    df = df.append(temp,ignore_index = True)

In [5]:
df.head()

,date,time,username,tweet,Food Delivery,Aspect,Sentiment,company
0,2020-08-23,16:30:40,pinkzaa_zaa,ใช้แต่ #LINEMAN มาตลอด\n______________________...,Yes,Company,Positive,LINEMAN
1,2020-08-23,11:29:02,roseapple110,มีคนเจอปัญหาไรเดอร์โทรหาไม่ติดตอนที่ปิดแอปทิ้ง...,Yes,Employee,Negative,LINEMAN
2,2020-08-22,18:58:12,twitneearoi,"เมนูโปรดของแมวอ้วนอย่างเรา วันนี้ขอเสนออออ ""ข้...",No,NaN,NaN,LINEMAN
3,2020-08-22,17:08:50,belloir_sj,ขั้นกว่าของการเลี้ยงหมี คือการอาบน้ำด้วย #แลคต...,No,NaN,NaN,LINEMAN
4,2020-08-22,16:23:15,ifiend_,แต่ #lineman โปรลดเข้าร่วมเยอะกว่านะ,Yes,Price,Positive,LINEMAN


## EDA

### Drop columns

In [6]:
df.drop(columns=["date","time","username"],inplace=True)

### Check Missing Value

In [7]:
np.sum(df.isnull())

tweet               0
Food Delivery       0
Aspect           3339
Sentiment        3340
company             0
dtype: int64

In [8]:
df.dropna(inplace=True)

In [9]:
df.reset_index(drop=True, inplace=True)

In [10]:
np.sum(df.isnull())

tweet            0
Food Delivery    0
Aspect           0
Sentiment        0
company          0
dtype: int64

## Preprocessing

In [11]:
## Add word in dict
custom_dict = set(thai_words())

System_ = 'ปิด ปิดร้าน ชั่วคราว ระบบ เงิน ตัด บัตร แคนเซิล ยกเลิก เป็น อาราย อะไร พัง ค้าง ล่ม เสีย สั่ง หา ไม่ได้ เด้ง รอนาน รอ ออเดอร์ พัฒนา ตัดเงิน ตัดบัตร หักเงิน แอป แอพ ปัก ปักหมุด พิกัด ใช้ ยาก ง่าย พัฒนา ไม่ตรง แผนที่ โอน เครดิต คืน รีบ แก้ ด่วน ขัดข้อง ฟื้น แก้ไข จ่ายเงิน ปัญหา ไม่เปิด ไม่เจอ แท็ก คืนเงิน ไม่พบ หา โดนระงับ ตรวจสอบ ขั้นตอน หักตังค์ เงินไม่เข้า เติมเงิน ไม่เข้า จ่ายไม่ได้ แผนที่ แบบใหม่ แบบเก่า แมพ พิกัด บัตรเครดิต บัค ฟีเจอร์ บั๊ก หมุน'
System_Aspect = System_.split(' ')

Company_ = 'คอลเซนเตอร์ คอลเซ็นเตอร์ รับผิดชอบ เบอร์ ติดต่อ สื่อสาร โทร เมล ยกเลิก เงินคืน ร้องเรียน ไม่ สนใจ เพิกเฉย รอสาย ตัดสาย บริการ ด่วน เมล โทร ศูนย์บริการ เจ้าหน้าที่ ปัญหา ชดเชย เลิกใช้ แบน บริษัท โกงเงิน บัญชี เลิกใช้ แคนเซิล ออเดอร์ ลบ องค์กร ระงับ เลิกสั่ง ประทับใจ โกง รีฟัน แนะนำ ชี้แจง แจ้ง รีวิว คอมเพลน'
Company_Aspect = Company_.split(' ')

Price_ =  'ราคา ค่าส่ง ส่วนลด โปรโมชั่น คุ้มค่า ไม่คุ้ม แพง ถูก ลด โปร บอกต่อ คุ้ม ค่าบริการ ใช้ไม่ได้ ใช้งานไม่ได้ คุณภาพ ใช้ได้ ซื้อ ขั้นต่ำ ส่งฟรี แกง บาท ส่วนต่าง ลูกค้า ใหม่ เก่า โค้ด เพิ่ม ฟรี ค่าอาหาร ค่าจอดรถ ต่ำกว่า รหัส ภาษี คำสั่งซื้อ เสียเงินเพิ่ม สังน้อย บวกเพิ่ม ขนาดเล็ก ค่าธรรมเนียม แถม เก็บ เล็ก คูปอง หมด เต็ม กด ทัน'

Price_Aspect = Price_.split(' ')

Employee_ = 'ยกเลิก รอ ช้า นาน ชั่วโมง ชม นาที ไม่ได้รับ ไม่ได้อาหาร ปัญหา รอเพิ่ม ออเดอร์ ผู้ส่ง พนักงาน คนขับ คนส่ง อบรม มารยาท คำพูด รอนาน บริการ นิสัย เละเทะ ไม่รับ สาย ส่ง ไรเดอร์ เสีย คน ติดต่อ ไม่ครบ ขาด หาย ช้า ไม่ครบ คนส่ง พนง สุภาพ นาที ไว ส่ง จัดส่ง ซ้อน ยังไม่ได้ ตั้งแต่ บอกทาง เหวี่ยง ส่งผิด ผิด ปฏิเสธ พี่ พูดเพราะ รับงาน ที่อยู่ หาย คุณ ไม่พอใจ น้อง คุ๊น ด่า เชค เช็ค ตรวจ สภาพ พ่วง ขอบคุณ โทร แซง คิว'

Employee_Aspect = Employee_.split(' ')

reviews_dict = System_Aspect+Company_Aspect+Price_Aspect+Employee_Aspect

for word in reviews_dict:
    custom_dict.add(word)

trie = dict_trie(dict_source= custom_dict)

In [12]:
def preprocessor(text):
    
    #remove hashtag
    hashtag_removed = re.sub(r"#\w+",'', text) 
    
    #remove url
    url_removed = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','', hashtag_removed)
    
    #word tokenize
    word_tokenized = pythainlp.word_tokenize(url_removed,custom_dict=trie,keep_whitespace=False,engine='pyicu')
    
    #thai filter
    word_thai = []
    for word in word_tokenized:
        if pythainlp.util.isthai(word):
            for punc in string.punctuation:
                word = word.replace(punc,'')
            if word != '':
                word_thai.append(word)
                
    #check vocab
    check_vocab = []
    for word in word_thai:
        if word in custom_dict:
            check_vocab.append(word)
                
    #remove stopword
    removed_stopword = []
    for word in check_vocab:
        if word not in pythainlp.corpus.common.thai_stopwords():
            removed_stopword.append(word)
    
    
    #remove name of company
    remove_company_name = []
    for word in removed_stopword:
        if word not in ['ฟู้ดแพนด้า','แกรป','แกป','ไลน์แมน','แพนด้า']:
            remove_company_name.append(word)
                
    return remove_company_name

## Naive Bayes

In [13]:
pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=preprocessor)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB(fit_prior=True)),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [14]:
X_train, X_test, y_train, y_test = train_test_split(df['tweet'], df[['Aspect', 'Sentiment']], test_size=0.2, random_state=0)

In [15]:
pipeline.fit(X_train, y_train['Aspect'])

Pipeline(memory=None,
         steps=[('bow',
                 CountVectorizer(analyzer=<function preprocessor at 0x1a23146320>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('classifier',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [16]:
y_predict = pipeline.predict(X_test)

In [35]:
print(classification_report(y_test['Aspect'],y_predict))
print(confusion_matrix(y_test['Aspect'],y_predict))

              precision    recall  f1-score   support

     Company       1.00      0.04      0.07       110
    Employee       0.58      0.88      0.70       224
       Price       0.74      0.95      0.83       248
      System       0.88      0.22      0.35       106

    accuracy                           0.67       688
   macro avg       0.80      0.52      0.49       688
weighted avg       0.75      0.67      0.59       688

[[  4  69  35   2]
 [  0 196  27   1]
 [  0  12 236   0]
 [  0  61  22  23]]


In [36]:
y_test['Aspect']

526     Employee
1771    Employee
1503       Price
3302     Company
979      Company
          ...   
764      Company
259     Employee
2539      System
1161     Company
347        Price
Name: Aspect, Length: 688, dtype: object

## Sentiment

In [39]:
models_raw = {}
#smote = SMOTE()

for aspect in ['Company','Employee','Price','System']:
    
    print('Aspect:', aspect)
    
    pipeline_raw = Pipeline([
        ('bow', CountVectorizer(analyzer=preprocessor)),  # strings to token integer counts
        ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
        ('classifier',MultinomialNB(fit_prior=True))])  # train on TF-IDF vectors 
    
    X_train_raw = X_train[y_train['Aspect'] == aspect]
    y_train_raw = y_train['Sentiment'][y_train['Aspect'] == aspect]
    print(len(X_train_raw), len(y_train_raw))
    
#     #Upsampling
#     X_train_raw_up, y_train_raw_up = smote.fit_sample(X_train_raw, y_train_raw)
#     print(len(X_train_raw_up), len(y_train_raw_up))
    
    pipeline_raw.fit(X_train_raw, y_train_raw)
    models_raw[aspect] = pipeline_raw
    
    y_predict_raw = pipeline_raw.predict(X_test[y_test['Aspect'] == aspect])
    
    print(classification_report(y_test['Sentiment'][y_test['Aspect'] == aspect],y_predict_raw))
    print(confusion_matrix(y_test['Sentiment'][y_test['Aspect'] == aspect],y_predict_raw))
    print('\n'*2)
    

Aspect: Company
433 433
              precision    recall  f1-score   support

    Negative       0.80      0.90      0.85        49
     Neutral       0.73      0.44      0.55        25
    Positive       0.75      0.83      0.79        36

    accuracy                           0.77       110
   macro avg       0.76      0.72      0.73       110
weighted avg       0.77      0.77      0.76       110

[[44  2  3]
 [ 7 11  7]
 [ 4  2 30]]



Aspect: Employee
891 891


/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

    Negative       0.71      1.00      0.83       155
     Neutral       0.00      0.00      0.00        32
    Positive       1.00      0.16      0.28        37

    accuracy                           0.72       224
   macro avg       0.57      0.39      0.37       224
weighted avg       0.66      0.72      0.62       224

[[155   0   0]
 [ 32   0   0]
 [ 31   0   6]]



Aspect: Price
1010 1010
              precision    recall  f1-score   support

    Negative       0.77      0.50      0.61        66
     Neutral       0.63      0.78      0.70       106
    Positive       0.73      0.70      0.71        76

    accuracy                           0.68       248
   macro avg       0.71      0.66      0.67       248
weighted avg       0.70      0.68      0.68       248

[[33 29  4]
 [ 7 83 16]
 [ 3 20 53]]



Aspect: System
418 418
              precision    recall  f1-score   support

    Negative       0.69      1.00      0.81    

/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
